In [ ]:
from IPython.display import clear_output
huggingface_cli_token = input("INSERT YOUR huggingface-cli_token:")
clear_output()

In [ ]:
!huggingface-cli login --token $huggingface_cli_token

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
The token `tirocinio2` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `tirocinio2`


In [ ]:
!pip install -q datasets --progress-bar off

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import pandas as pd
import pickle

In [ ]:
experiment_name = "fine-tuning-output-global"

In [ ]:
from google.colab import drive
from IPython.display import Image, display

mount_point = "/content/drive"
base_path = mount_point + "/MyDrive"

input_data_path = base_path + "/output_global_V3_DDD.csv"    #Cambia il percorso con il nome del file di training su cui desideri addestrare

output_data_path = base_path + experiment_name
log_filename = base_path + experiment_name + ".log"

drive.mount(mount_point, force_remount=True)

Mounted at /content/drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%%capture
%pip install -U bitsandbytes
%pip install -U transformers
%pip install -U peft
%pip install -U accelerate
%pip install -U trl

In [ ]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import (
    LoraConfig,
    PeftModel,
    prepare_model_for_kbit_training,
    get_peft_model,
)
import os, torch
from datasets import load_dataset
from trl import SFTTrainer


In [ ]:
base_model = "mistralai/Mistral-7B-Instruct-v0.3"
new_model = "Mistral-7B-Instruct-v0.3-fine-tuning-global-DDD-alternativePrompt"    #Cambia il nome del modello fine-tunato a tuo piacimento

In [ ]:
def summarize_adherence_columns(row):
    """
    Summarize the three adherence columns into a single value representing which one is active
    0 for low adherence, 1 for intermediate adherence and 2 for high adherence
    :param row: dataframe row
    :return: adherence value
    """

    if row["BASSA ADERENZA"] == 1:
        return 0

    if row["INTERMEDIA ADERENZA"] == 1:
        return 1

    if row["ALTA ADERENZA"] == 1:
        return 2

    return -1

In [ ]:
data = data = pd.read_csv(input_data_path,
        parse_dates=["DT_NAS"],
        date_format="%d/%m/%Y",
        usecols=[
            # Sex
            "SESSO",
            # Date of birth
            "DT_NAS",
            # City of birth
            "COMUNE NASCITA",
            # City of residence
            "COMUNE_RESIDENZA",
            # First drug to take
            "PRIMO_PROD",
            # Shift
            "SHIFT",
            # Adherence
            "BASSA ADERENZA",
            "INTERMEDIA ADERENZA",
            "ALTA ADERENZA",
            # Follow-up persistence
            "Persistenza di Follow-up",
        ],)

# Convert adherence into a single value: 0 for low, 1 for middle, 2 for high, and then remove these columns
data["ADERENZA"] = data.apply(summarize_adherence_columns, axis=1)
data.drop(columns=["BASSA ADERENZA", "INTERMEDIA ADERENZA", "ALTA ADERENZA"], inplace=True)

data.head(3)

,SESSO,DT_NAS,COMUNE NASCITA,COMUNE_RESIDENZA,SHIFT,Persistenza di Follow-up,PRIMO_PROD,ADERENZA
0,F,1966-10-01,ASSEMINI,ASSEMINI,2,1,1480002,2
1,F,1963-02-11,CAGLIARI,QUARTUCCIU,1,1,1480022,2
2,F,1976-12-27,CAGLIARI,CAGLIARI,1,0,1480022,2


In [ ]:
data.shape[0]

162

In [ ]:
def apply_fine_tuning_template(data):


  template = f"""<s>[INST]Considering the sex, birth date, birth city, residence city, first drug to take and shift of a
    patient, all information given line by line and formatted as 'label: value', the whole block of lines being enclosed
    by triple single quotes, predict the values for adherence and follow-up persistence.
    Do not consider any information than those provided enclosed by triple single quotes. Your task is to predict one value for adherence and one value for follow-up persistence based on the patient's information and to output the predicted
    values in the same format.
    Do not absolutely include for any reason any other content, especially input information, in the output.
    '''
    sex: {data['SESSO']}
    birth date: {data['DT_NAS']}
    birth city: {data['COMUNE NASCITA']}
    residence city: {data['COMUNE_RESIDENZA']}
    first drug to take: {data['PRIMO_PROD']}
    shift: {data['SHIFT']}
    '''[/INST]
    adherence: {data['ADERENZA']}
    persistence follow-up: {data['Persistenza di Follow-up']}
    </s>
    """

  return template


In [ ]:
data['fine_tuning_prompt'] = data.apply(apply_fine_tuning_template, axis=1)

In [ ]:
data.head(3)

,SESSO,DT_NAS,COMUNE NASCITA,COMUNE_RESIDENZA,SHIFT,Persistenza di Follow-up,PRIMO_PROD,ADERENZA,fine_tuning_prompt
0,F,1966-10-01,ASSEMINI,ASSEMINI,2,1,1480002,2,<s>[INST]\n This is a classification task. ...
1,F,1963-02-11,CAGLIARI,QUARTUCCIU,1,1,1480022,2,<s>[INST]\n This is a classification task. ...
2,F,1976-12-27,CAGLIARI,CAGLIARI,1,0,1480022,2,<s>[INST]\n This is a classification task. ...


In [ ]:
data.iloc[0]['fine_tuning_prompt']

"<s>[INST]\n    This is a classification task. Your goal is to predict the values for adherence and follow-up persistence  based on the provided patient information.\n\n    Use only the data enclosed in triple single quotes below. Do not use external information.\n    Data is provided in the format 'label: value', with one piece of information per line.\n\n    Task: Predict the following categories:\n    - adherence: One of ['0', '1', '2']\n    - follow-up persistence: One of ['0', '1']\n\n    Patient information:\n    '''\n    sex: F\n    birth date: 1966-10-01 00:00:00\n    birth city: ASSEMINI\n    residence city: ASSEMINI\n    first drug to take: 1480002\n    shift: 2\n    '''\n\n    [/INST]\n    Expected Output:\n    adherence: 2\n    follow-up persistence: 1\n    </s>\n    "

In [ ]:
from datasets import Dataset

# Funzione per convertire il DataFrame in un formato compatibile con load_dataset
def convert_to_dataset(df):
    dataset_dict = {}
    for column in df.columns:
        dataset_dict[column] = df[column].tolist()

    dataset = Dataset.from_dict(dataset_dict)
    return dataset

In [ ]:
dataset = convert_to_dataset(data[['fine_tuning_prompt']])

In [ ]:
dataset

Dataset({
    features: ['fine_tuning_prompt'],
    num_rows: 162
})

In [ ]:
# Load base model(Zephyr-7B 7B)
bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
)
model = AutoModelForCausalLM.from_pretrained(
        base_model,
        #load_in_4bit=True,
        quantization_config=bnb_config,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True,
)
model.config.use_cache = False # silence the warnings. Please re-enable for inference!
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.add_bos_token, tokenizer.add_eos_token


config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

(True, True)

In [ ]:
model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['up_proj', 'base_layer', 'down_proj']
)
model = get_peft_model(model, peft_config)

In [ ]:
#Hyperparamter
training_arguments = TrainingArguments(
    output_dir="./results",
    num_train_epochs=100,     #Numero max di epoche
    per_device_train_batch_size=14, #4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=1000,      #Numero max di step
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    #report_to="wandb"
)


In [ ]:
# Setting sft parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    max_seq_length= 2048,
    dataset_text_field="fine_tuning_prompt",
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length, dataset_text_field. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/162 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
trainer.train()

# Save the fine-tuned model
trainer.model.save_pretrained("/content/drive/MyDrive/" + new_model)
# wandb.finish()
model.config.use_cache = True
model.eval()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
25,0.589800
50,0.172600
75,0.144300
100,0.140900
125,0.136600
150,0.132900
175,0.124600
200,0.117600
225,0.111200
250,0.102700


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32768, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralSdpaAttention(
              (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
              (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
              (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
              (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
              (rotary_emb): MistralRotaryEmbedding()
            )
            (mlp): MistralMLP(
              (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
              (up_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=14336, bias=False)
                (lora_dropout): ModuleDict(
                  (def